In [35]:
import tensorflow as tf
print(tf.test.is_gpu_available())
import pandas as pd
df=pd.read_csv('data/moviedialog.csv')

True


In [36]:
df.head()

,Unnamed: 0,statement,reply
0,0,you're asking me out. that's so cute. what's y...,forget it.
1,1,"no, no, it's my fault we didn't have a proper ...",cameron.
2,2,"gosh, if only we could find kat a boyfriend...",let me see what i can do.
3,3,c'esc ma tete. this is my head,right. see? you're ready for the quiz.
4,4,how is our little find the wench a date plan p...,"well, there's someone i think might be"


In [37]:
input_texts,target_texts=[],[]
input_vocabulary=set()
output_vocabulary=set()
start_token='\t'
stop_token='\n'
max_training_samples=min(25000,len(df)-1)

In [38]:
for input_text,target_text in zip(df.statement,df.reply):
    target_text=start_token+target_text+stop_token
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_vocabulary:
            input_vocabulary.add(char)
    for char in target_text:
        if char not in output_vocabulary:
            output_vocabulary.add(char)
            

In [39]:
input_vocabulary=sorted(input_vocabulary)
output_vocabulary=sorted(output_vocabulary)
input_vocab_size=len(input_vocabulary)
output_vocab_size=len(output_vocabulary)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])
input_token_index=dict([(char,i ) for i,char in enumerate(input_vocabulary)])
target_token_index=dict([(char,i) for i,char in enumerate(output_vocabulary)])
reverse_input_token_index=dict([(i,char) for i,char in enumerate(input_vocabulary)])
reverse_target_token_index=dict([(i,char) for i,char in enumerate(output_vocabulary)])


In [40]:
import numpy as np
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,input_vocab_size),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,output_vocab_size),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,output_vocab_size),dtype='float32')


In [41]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1
    for yt,chart in enumerate(target_text):
        decoder_input_data[i,yt,target_token_index[chart]]=1
        if yt>0:
            decoder_target_data[i,yt-1,target_token_index[chart]]=1


In [ ]:
import numpy as np
def onehot_encode_ger(sentence,char_indices,maxlen=40):
    X=np.zeros((maxlen,len(char_indices.keys())))
    for t,char in enumerate(sentence):
        X[t,char_indices[char]]=1
    return X
    
def onehot_encode_label(char,char_indices):
    Y=np.zeros((len(char_indices)))
    Y[char_indices[char]]=1
    return Y

In [ ]:

def data_generator(datastore,next_char,char_indices,batchsize=32,maxlen=40):
    X,Y,Z=[],[],[]
    while True:
        for i in range(len(datastore)):
            if (i % batchsize==0 and X and Y) or (i==len(datastore)):
                X=np.reshape(X,(len(X),maxlen,len(char_indices)))
                Y=np.reshape(Y,(len(Y),len(char_indices)))
                yield X,Y
                X,Y=[],[]
            x=datastore[i]
            y=next_chars[i]
            x=onehot_encode_ger(x,char_indices,maxlen=maxlen)
            y=onehot_encode_label(y,char_indices)   
            X.append(x)
            Y.append(y)

In [43]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM,Dense,Input
batch_size=256
epochs=100
num_neurons=256
encoder_inputs=Input(shape=(None,input_vocab_size))
encoder=LSTM(num_neurons,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

decoder_inputs=Input(shape=(None,output_vocab_size))
decoder_lstm=LSTM(num_neurons,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(output_vocab_size,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics='acc')
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.1)

Epoch 1/100
227/227 [==============================] - 10s 38ms/step - loss: 1.1074 - acc: 0.0664 - val_loss: 1.0865 - val_acc: 0.0626
Epoch 2/100
227/227 [==============================] - 8s 35ms/step - loss: 1.0683 - acc: 0.0632 - val_loss: 1.0863 - val_acc: 0.0661
Epoch 3/100
227/227 [==============================] - 8s 35ms/step - loss: 1.0439 - acc: 0.0681 - val_loss: 1.0070 - val_acc: 0.0808
Epoch 4/100
227/227 [==============================] - 8s 35ms/step - loss: 1.0212 - acc: 0.0728 - val_loss: 1.0251 - val_acc: 0.0778
Epoch 5/100
227/227 [==============================] - 8s 35ms/step - loss: 0.9911 - acc: 0.0773 - val_loss: 1.0262 - val_acc: 0.0780
Epoch 6/100
227/227 [==============================] - 8s 35ms/step - loss: 0.9767 - acc: 0.0806 - val_loss: 1.0173 - val_acc: 0.0803
Epoch 7/100
227/227 [==============================] - 8s 35ms/step - loss: 0.9632 - acc: 0.0839 - val_loss: 0.9588 - val_acc: 0.0924
Epoch 8/100
227/227 [==============================] - 8s 35m

In [44]:
encoder_model=Model(encoder_inputs,encoder_states)
thought_input=[Input(shape=(num_neurons,)),Input(shape=(num_neurons,))]
decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=thought_input)
decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)
decoder_model=Model(inputs=[decoder_inputs]+thought_input,outputs=[decoder_outputs]+decoder_states)


In [45]:
def decode_sequence(input_seq):
    thought=encoder_model.predict(input_seq)
    target_seq=np.zeros((1,1,output_vocab_size))
    target_seq[0,0,target_token_index[stop_token]]=1
    stop_condition=False
    generated_sequence=''
    while not stop_condition:
        output_tokens,h,c=decoder_model.predict([target_seq]+thought)
        generated_token_idx=np.argmax(output_tokens[0,-1,:])
        generated_char=reverse_target_token_index[generated_token_idx]
        generated_sequence+=generated_char
        if (generated_char==stop_token or len(generated_sequence)>max_decoder_seq_length):
            stop_condition=True
        target_seq=np.zeros((1,1,output_vocab_size))
        target_seq[0,0,generated_token_idx]=1
        thought=[h,c]
    return generated_sequence


In [46]:
def response(input_text):
    input_seq=np.zeros((1,max_decoder_seq_length,input_vocab_size),dtype='float32')
    for i ,char in enumerate(input_text):
        input_seq[0,t,input_token_index[char]]=1
    decode_sentence=decode_sequence(input_seq)
    print('Bot reply:'+decode_sentence)


In [47]:
response("what is the interest?")

Bot reply:it you want to see you to the proble the stare the stare the stare the stare the stare the stare the st


In [48]:
response("why?")

Bot reply:it you want to see you to the proble the stare the stare the stare the stare the stare the stare the st


In [49]:
response("do you like coffee?")

Bot reply:it you want to see you to the proble the stare the stare the stare the stare the stare the stare the st


In [50]:
response("do you like football?")

Bot reply:it you want to see you to the proble the stare the stare the stare the stare the stare the stare the st


In [51]:
response("do you like me?")

Bot reply:it you want to see you to the proble the stare the stare the stare the stare the stare the stare the st
